## Trayectoría


In [1]:
import numpy as np
import mujoco
from collections import namedtuple
import matplotlib.pyplot as plt


In [5]:
def joint_trajectory(q0, qf, steps=100):
    """
    Genera una trayectoria de joints usando polinomio quintico (solo MuJoCo)
    
    Args:
        q0: configuración inicial de joints [q1, q2, ..., qn]
        qf: configuración final de joints [q1, q2, ..., qn]
        steps: número de pasos en la trayectoria
    
    Returns:
        Objeto con atributo .q que contiene el array de configuraciones
    """
    q0 = np.array(q0)
    qf = np.array(qf)
    
    # Verificar dimensiones
    if len(q0) != len(qf):
        raise ValueError("q0 y qf deben tener la misma longitud")
    
    n_joints = len(q0)
    
    # Crear vector de tiempo normalizado (0 a 1)
    t = np.linspace(0, 1, steps)
    
    # Condiciones de frontera para polinomio quintico
    # posiciones: q0 -> qf
    # velocidades: 0 -> 0 (inicio y final en reposo)
    # aceleraciones: 0 -> 0
    
    # Matriz para resolver coeficientes del polinomio quintico
    # q(t) = a5*t^5 + a4*t^4 + a3*t^3 + a2*t^2 + a1*t + a0
    # q'(t) = 5*a5*t^4 + 4*a4*t^3 + 3*a3*t^2 + 2*a2*t + a1
    # q''(t) = 20*a5*t^3 + 12*a4*t^2 + 6*a3*t + 2*a2
    
    # Condiciones: q(0)=q0, q(1)=qf, q'(0)=0, q'(1)=0, q''(0)=0, q''(1)=0
    A = np.array([
        [0, 0, 0, 0, 0, 1],      # q(0) = q0
        [1, 1, 1, 1, 1, 1],      # q(1) = qf  
        [0, 0, 0, 0, 1, 0],      # q'(0) = 0
        [5, 4, 3, 2, 1, 0],      # q'(1) = 0
        [0, 0, 0, 2, 0, 0],      # q''(0) = 0
        [20, 12, 6, 2, 0, 0]     # q''(1) = 0
    ])
    
    # Resolver para cada joint
    trajectories = []
    
    for i in range(n_joints):
        # Vector de condiciones para este joint
        b = np.array([q0[i], qf[i], 0, 0, 0, 0])
        
        # Resolver coeficientes del polinomio
        coeffs = np.linalg.solve(A, b)
        
        # Evaluar polinomio para todos los tiempos
        joint_traj = np.polyval(coeffs, t)
        trajectories.append(joint_traj)
    
    # Transponer para tener filas = tiempo, columnas = joints
    q_array = np.array(trajectories).T
    
    # Crear estructura de retorno compatible
    TrayectoriaResultado = namedtuple('TrayectoriaResultado', ['q'])
    return TrayectoriaResultado(q=q_array)

def joint_trajectory_via_points(waypoints, steps_per_segment=50):
    """
    Genera trayectoria pasando por múltiples puntos via
    
    Args:
        waypoints: lista de configuraciones [[q1], [q2], ..., [qn]]
        steps_per_segment: pasos entre cada waypoint
    
    Returns:
        Objeto con atributo .q que contiene el array de configuraciones
    """
    if len(waypoints) < 2:
        raise ValueError("Se necesitan al menos 2 waypoints")
    
    waypoints = [np.array(wp) for wp in waypoints]
    
    # Generar trayectoria entre cada par de waypoints
    all_trajectories = []
    
    for i in range(len(waypoints) - 1):
        q0 = waypoints[i]
        qf = waypoints[i + 1]
        
        # Generar segmento
        segment = joint_trajectory(q0, qf, steps_per_segment)
        
        # Evitar duplicar puntos en las uniones (excepto el primero)
        if i == 0:
            all_trajectories.append(segment.q)
        else:
            all_trajectories.append(segment.q[1:])  # Saltar primer punto
    
    # Concatenar todos los segmentos
    q_total = np.vstack(all_trajectories)
    
    TrayectoriaResultado = namedtuple('TrayectoriaResultado', ['q'])
    return TrayectoriaResultado(q=q_total)

def execute_trajectory_mujoco(model, data, trajectory, joint_names=None, dt=0.01):
    """
    Ejecuta una trayectoria en MuJoCo
    
    Args:
        model: modelo de MuJoCo
        data: datos de MuJoCo  
        trajectory: resultado de joint_trajectory()
        joint_names: lista de nombres de joints (opcional)
        dt: paso de tiempo para la simulación
    """
    if joint_names is None:
        # Usar todos los joints controlables
        joint_names = [
            "shoulder_pan_joint",
            "shoulder_lift_joint", 
            "elbow_joint",
            "wrist_1_joint",
            "wrist_2_joint",
            "wrist_3_joint"
        ]
    
    # Obtener IDs de los joints
    joint_ids = []
    for name in joint_names:
        try:
            joint_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, name)
            joint_ids.append(joint_id)
        except:
            print(f"Warning: Joint {name} no encontrado")
    
    # Ejecutar trayectoria
    for i, q_target in enumerate(trajectory.q):
        # Establecer posiciones objetivo
        for j, joint_id in enumerate(joint_ids):
            if j < len(q_target):
                data.ctrl[joint_id] = q_target[j]
        
        # Paso de simulación
        mujoco.mj_step(model, data)
        
        print(f"Paso {i+1}/{len(trajectory.q)}: {np.round(q_target, 3)}")

# ===== EJEMPLO DE USO =====
if __name__ == "__main__":
    # Configuraciones de ejemplo
    q1 = np.array([0, -1.45, -0.88, 0.723, -1.51, -1.51])
    q2 = np.array([0.5, -1.0, -0.5, 0.5, -1.0, -1.0])
    
    # Generar trayectoria simple
    print("=== TRAYECTORIA SIMPLE ===")
    qt = joint_trajectory(q1, q2, 10)
    print(f"Forma del array: {qt.q.shape}")
    print("Primeras configuraciones:")
    for i in range(5):
        print(f"  Paso {i+1}: {np.round(qt.q[i], 3)}")
    
    # Trayectoria con puntos via
    print("\n=== TRAYECTORIA CON WAYPOINTS ===")
    waypoints = [
        [0, -1.5, -1.5, 1.5, -1.5, 0],      # Posición home
        [0.5, -1.0, -0.5, 0.5, -1.0, -1.0], # Posición intermedia
        [0, -1.45, -0.88, 0.723, -1.51, -1.51] # Posición final
    ]
    
    qt_via = joint_trajectory_via_points(waypoints, 20)
    print(f"Forma del array: {qt_via.q.shape}")
    print("Configuraciones en los waypoints:")
    print(f"  Inicio: {np.round(qt_via.q[0], 3)}")
    print(f"  Medio: {np.round(qt_via.q[len(qt_via.q)//2], 3)}")
    print(f"  Final: {np.round(qt_via.q[-1], 3)}")

=== TRAYECTORIA SIMPLE ===
Forma del array: (10, 6)
Primeras configuraciones:
  Paso 1: [ 0.    -1.45  -0.88   0.723 -1.51  -1.51 ]
  Paso 2: [ 0.006 -1.445 -0.876  0.72  -1.504 -1.504]
  Paso 3: [ 0.038 -1.416 -0.851  0.706 -1.471 -1.471]
  Paso 4: [ 0.105 -1.356 -0.8    0.676 -1.403 -1.403]
  Paso 5: [ 0.198 -1.271 -0.729  0.635 -1.308 -1.308]

=== TRAYECTORIA CON WAYPOINTS ===
Forma del array: (39, 6)
Configuraciones en los waypoints:
  Inicio: [ 0.  -1.5 -1.5  1.5 -1.5  0. ]
  Medio: [ 0.5 -1.  -0.5  0.5 -1.  -1. ]
  Final: [ 0.    -1.45  -0.88   0.723 -1.51  -1.51 ]


In [23]:
configuraciones_puntos = [
    [1.5, -1.07, -0.88, 2.01, -2.58, -1.63, -0.628],   
    [1.5, -1.32, -1.45, 2.01, -2.2, -1.76, -2.64],     
    [-0.615, -1.32, -1.45, 2.01, -2.2, -1.76, -2.64],   
    [-0.615, 0.628, -1.45, 2.01, -2.2, -1.76, -2.64],   
    [-0.615, 0.628, -1.07, 2.01, -2.58, -1.63, -4.34],  
    [-0.615, 0.628, -1.45, 2.01, -2.2, -1.76, -2.64], 
    [-0.615, -1.32, -1.45, 2.01, -2.2, -1.76, -2.64], 
    [1.5, -1.32, -1.45, 2.01, -2.2, -1.76, -2.64],
    [1.5, -1.07, -0.88, 2.01, -2.58, -1.63, -0.628], 
    [1.5, -1.07, -0.88, 2.01, -2.58, -1.63, -0.628],   
    [1.5, -1.32, -1.45, 2.01, -2.2, -1.76, -2.64],     
    [-0.615, -1.32, -1.45, 2.01, -2.2, -1.76, -2.64],   
    [-0.615, 0.628, -1.45, 2.01, -2.2, -1.76, -2.64],   
    [-0.615, 0.628, -1.07, 2.01, -2.58, -1.63, -4.34],  
    [-0.615, 0.628, -1.45, 2.01, -2.2, -1.76, -2.64], 
    [-0.615, -1.32, -1.45, 2.01, -2.2, -1.76, -2.64], 
    [1.5, -1.32, -1.45, 2.01, -2.2, -1.76, -2.64],
    [1.5, -1.07, -0.88, 2.01, -2.58, -1.63, -0.628]
]

In [24]:
q = joint_trajectory_via_points(configuraciones_puntos , steps_per_segment=50)
print(q)

TrayectoriaResultado(q=array([[ 1.5       , -1.07      , -0.88      , ..., -2.58      ,
        -1.63      , -0.628     ],
       [ 1.5       , -1.0700206 , -0.88004698, ..., -2.57996868,
        -1.63001071, -0.62816582],
       [ 1.5       , -1.07015976, -0.88036425, ..., -2.57975717,
        -1.63008307, -0.62928574],
       ...,
       [ 1.5       , -1.07015976, -0.88036425, ..., -2.57975717,
        -1.63008307, -0.62928574],
       [ 1.5       , -1.0700206 , -0.88004698, ..., -2.57996868,
        -1.63001071, -0.62816582],
       [ 1.5       , -1.07      , -0.88      , ..., -2.58      ,
        -1.63      , -0.628     ]]))


In [25]:
print(len(q.q))

834


In [15]:
import mujoco
import mujoco.viewer
import numpy as np
import time

def ejecutar_trayectoria(model, data, trayectoria_q, duracion_total=10.0, mostrar_progreso=True):
    """
    Ejecuta una trayectoria de configuraciones q en MuJoCo
    
    Args:
        model: modelo de MuJoCo
        data: datos de MuJoCo
        trayectoria_q: lista de configuraciones [[q0], [q1], [q2], ...]
        duracion_total: duración total de la trayectoria en segundos
        mostrar_progreso: si mostrar información de progreso
    """
    print(f"🎬 Ejecutando trayectoria con {len(trayectoria_q)} puntos")
    print(f"⏱️  Duración total: {duracion_total:.1f} segundos")
    
    # Nombres de los joints en orden
    joint_names = [
        "linear_joint",
        "shoulder_pan_joint",
        "shoulder_lift_joint", 
        "elbow_joint",
        "wrist_1_joint",
        "wrist_2_joint",
        "wrist_3_joint"
    ]
    
    # PASO 1: Configurar posición inicial (q[0])
    print(f"📍 Configurando posición inicial: {np.round(trayectoria_q[0], 3)}")
    
    # Resetear datos
    mujoco.mj_resetData(model, data)
    
    # Establecer posición inicial
    for i, joint_name in enumerate(joint_names):
        if i < len(trayectoria_q[0]):
            joint_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, joint_name)
            data.qpos[joint_id] = trayectoria_q[0][i]
            data.ctrl[i] = trayectoria_q[0][i]  # También establecer control
    
    # Actualizar simulación
    mujoco.mj_forward(model, data)
    
    # Verificar posición inicial del efector
    efector_pos = data.body('wrist_3_link').xpos
    print(f"🎯 Posición inicial del efector: {np.round(efector_pos, 3)}")
    
    # PASO 2: Ejecutar trayectoria con viewer
    tiempo_entre_puntos = duracion_total / (len(trayectoria_q) - 1)
    print(f"⏰ Tiempo entre puntos: {tiempo_entre_puntos:.2f} segundos")
    
    with mujoco.viewer.launch_passive(model, data) as viewer:
        punto_actual = 0
        tiempo_inicio = time.time()
        tiempo_ultimo_punto = tiempo_inicio
        
        print(f"🚀 Iniciando ejecución de trayectoria...")
        
        while viewer.is_running() and punto_actual < len(trayectoria_q):
            tiempo_transcurrido = time.time() - tiempo_inicio
            
            # Calcular qué punto de la trayectoria usar
            indice_objetivo = min(int(tiempo_transcurrido / tiempo_entre_puntos), len(trayectoria_q) - 1)
            
            # Si es tiempo de cambiar al siguiente punto
            if indice_objetivo > punto_actual:
                punto_actual = indice_objetivo
                
                if mostrar_progreso:
                    progreso = (punto_actual / (len(trayectoria_q) - 1)) * 100
                    efector_pos = data.body('wrist_3_link').xpos
                    print(f"📍 Punto {punto_actual+1}/{len(trayectoria_q)} ({progreso:.1f}%) - "
                          f"Efector: {np.round(efector_pos, 3)}")
                
                # Aplicar nueva configuración
                for i, joint_name in enumerate(joint_names):
                    if i < len(trayectoria_q[punto_actual]):
                        joint_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, joint_name)
                        data.ctrl[i] = trayectoria_q[punto_actual][i]
            
            # Actualizar simulación
            mujoco.mj_step(model, data)
            viewer.sync()
            time.sleep(0.01)  # Control de velocidad
        
        # Mantener posición final por un momento
        if viewer.is_running():
            print(f"🏁 Trayectoria completada. Manteniendo posición final...")
            efector_pos_final = data.body('wrist_3_link').xpos
            print(f"🎯 Posición final del efector: {np.round(efector_pos_final, 3)}")
            
            # Mantener por 3 segundos adicionales
            tiempo_final = time.time()
            while viewer.is_running() and (time.time() - tiempo_final) < 3.0:
                mujoco.mj_step(model, data)
                viewer.sync()
                time.sleep(0.01)

def ejecutar_trayectoria_interpolada(model, data, trayectoria_q, duracion_total=10.0, interpolar=True):
    """
    Ejecuta una trayectoria con interpolación suave entre puntos
    
    Args:
        model: modelo de MuJoCo
        data: datos de MuJoCo
        trayectoria_q: lista de configuraciones
        duracion_total: duración total en segundos
        interpolar: si interpolar suavemente entre puntos
    """
    print(f"🎬 Ejecutando trayectoria INTERPOLADA con {len(trayectoria_q)} puntos")
    
    joint_names = [
        "linear_joint", "shoulder_pan_joint", "shoulder_lift_joint", 
        "elbow_joint", "wrist_1_joint", "wrist_2_joint", "wrist_3_joint"
    ]
    
    # Configurar posición inicial
    mujoco.mj_resetData(model, data)
    for i, joint_name in enumerate(joint_names):
        if i < len(trayectoria_q[0]):
            joint_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, joint_name)
            data.qpos[joint_id] = trayectoria_q[0][i]
            data.ctrl[i] = trayectoria_q[0][i]
    mujoco.mj_forward(model, data)
    
    with mujoco.viewer.launch_passive(model, data) as viewer:
        tiempo_inicio = time.time()
        
        while viewer.is_running():
            tiempo_transcurrido = time.time() - tiempo_inicio
            
            if tiempo_transcurrido >= duracion_total:
                break
            
            # Calcular progreso (0.0 a 1.0)
            progreso = tiempo_transcurrido / duracion_total
            
            if interpolar:
                # Interpolación suave a lo largo de toda la trayectoria
                posicion_real = progreso * (len(trayectoria_q) - 1)
                indice_inferior = int(posicion_real)
                indice_superior = min(indice_inferior + 1, len(trayectoria_q) - 1)
                factor_interpolacion = posicion_real - indice_inferior
                
                # Interpolar entre dos puntos consecutivos
                if indice_inferior < len(trayectoria_q) and indice_superior < len(trayectoria_q):
                    q_interpolada = []
                    for j in range(len(trayectoria_q[0])):
                        q_inf = trayectoria_q[indice_inferior][j]
                        q_sup = trayectoria_q[indice_superior][j]
                        q_actual = q_inf + factor_interpolacion * (q_sup - q_inf)
                        q_interpolada.append(q_actual)
                    
                    # Aplicar configuración interpolada
                    for i, joint_name in enumerate(joint_names):
                        if i < len(q_interpolada):
                            joint_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, joint_name)
                            data.ctrl[i] = q_interpolada[i]
            else:
                # Sin interpolación - saltos discretos
                indice_actual = min(int(progreso * len(trayectoria_q)), len(trayectoria_q) - 1)
                for i, joint_name in enumerate(joint_names):
                    if i < len(trayectoria_q[indice_actual]):
                        joint_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, joint_name)
                        data.ctrl[i] = trayectoria_q[indice_actual][i]
            
            # Mostrar progreso cada segundo
            if int(tiempo_transcurrido) != int(tiempo_transcurrido - 0.02):
                efector_pos = data.body('wrist_3_link').xpos
                print(f"⏱️  {tiempo_transcurrido:.1f}s ({progreso*100:.1f}%) - "
                      f"Efector: {np.round(efector_pos, 3)}")
            
            mujoco.mj_step(model, data)
            viewer.sync()
            time.sleep(0.02)

def crear_trayectoria_ejemplo():
    """Crea una trayectoria de ejemplo con las configuraciones proporcionadas"""
    trayectoria = [
        [1.5, -1.07, -0.88, 2.01, -2.58, -1.63, -0.628],    # Punto 1
        [1.5, -1.32, -1.45, 2.01, -2.2, -1.76, -2.64],      # Punto 2
        [-0.615, -1.32, -1.45, 2.01, -2.2, -1.76, -2.64],   # Punto 3
        [-0.615, 0.628, -1.45, 2.01, -2.2, -1.76, -2.64],   # Punto 4
        [-0.615, 0.628, -1.07, 2.01, -2.58, -1.63, -4.34]   # Punto 5
    ]
    return trayectoria

In [27]:
model = mujoco.MjModel.from_xml_path("wherehouse_system.xml")
data = mujoco.MjData(model)
ejecutar_trayectoria(model, data, q.q,duracion_total=120,mostrar_progreso=True)


🎬 Ejecutando trayectoria con 834 puntos
⏱️  Duración total: 120.0 segundos
📍 Configurando posición inicial: [ 1.5   -1.07  -0.88   2.01  -2.58  -1.63  -0.628]
🎯 Posición inicial del efector: [0.986 0.577 0.623]
⏰ Tiempo entre puntos: 0.14 segundos
🚀 Iniciando ejecución de trayectoria...
📍 Punto 2/834 (0.1%) - Efector: [0.986 0.577 0.621]
📍 Punto 3/834 (0.2%) - Efector: [0.986 0.576 0.619]
📍 Punto 4/834 (0.4%) - Efector: [0.986 0.576 0.617]
📍 Punto 5/834 (0.5%) - Efector: [0.987 0.576 0.615]
📍 Punto 6/834 (0.6%) - Efector: [0.987 0.576 0.614]
📍 Punto 7/834 (0.7%) - Efector: [0.987 0.576 0.614]
📍 Punto 8/834 (0.8%) - Efector: [0.987 0.576 0.614]
📍 Punto 9/834 (1.0%) - Efector: [0.988 0.576 0.615]
📍 Punto 10/834 (1.1%) - Efector: [0.989 0.577 0.616]
📍 Punto 11/834 (1.2%) - Efector: [0.99  0.578 0.619]
📍 Punto 12/834 (1.3%) - Efector: [0.991 0.579 0.622]
📍 Punto 13/834 (1.4%) - Efector: [0.992 0.581 0.625]
📍 Punto 14/834 (1.6%) - Efector: [0.994 0.583 0.63 ]
📍 Punto 15/834 (1.7%) - Efector